In [1]:
import requests
import time
import random
import json
import os
import sqlite3

In [2]:
diretorio_dados = f'{os.path.abspath("../dados")}{os.path.sep}'

In [3]:
 def baixar_dados_episodios():
    TEMPO_MINIMO=3.0
    TEMPO_MAXIMO=15.0

    resultados = list()
    url_inicial = 'https://api.simplecast.com/podcasts/ab2964e7-bcad-4f2f-9698-45cb681f0d69/episodes?limit=15&offset=0&private=false&sort=desc&status=published'
    headers = {
        'authority': 'api.simplecast.com',
        'accept': 'application/json, text/plain, */*',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',
        'dnt': '1',
        'sec-fetch-site': 'same-site',
        'sec-fetch-mode': 'cors',
        'sec-fetch-dest': 'empty',    
        'origin': 'https://naruhodo-615be80d.simplecast.com',
        'referer': 'https://naruhodo-615be80d.simplecast.com/',
        'accept-language': 'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7'
    }
    requisicao = requests.get(url_inicial, headers=headers)
    resultado = requisicao.json()
    resultados.extend(resultado['collection'])
    print(f'Página 1 de {resultado["pages"]["total"]}.')

    while resultado['pages']['next'] != None:
        url = resultado['pages']['next']['href']
        soneca = random.uniform(TEMPO_MINIMO, TEMPO_MAXIMO)
        print(f'Sleep {soneca:.2f}')
        time.sleep(soneca)

        requisicao = requests.get(url, headers=headers)
        resultado = requisicao.json()
        resultados.extend(resultado['collection'])
        print(f'Página {resultado["pages"]["current"]} de {resultado["pages"]["total"]}.')
    
    return resultados

In [4]:
def salvar_dados_episodios(dados):
    with open(f'{diretorio_dados}episodios.json', 'w') as arquivo:
        json.dump(dados, arquivo)

In [5]:
def executar():
    with sqlite3.connect(f'{diretorio_dados}dados_episodios.db') as conexao:
        conexao.execute('''
        CREATE TABLE IF NOT EXISTS episodio (
            id TEXT NOT NULL PRIMARY KEY,
            numero INTEGER,
            titulo TEXT,
            descricao TEXT,
            duracao INTEGER,
            pulicacao DATETIME
        );
        ''')
        
        episodeos = baixar_dados_episodios()
        salvar_dados_episodios(episodeos)
        
        existentes = {item[0] for item in conexao.execute('select id from episodio').fetchall()}
        
        for episodio in episodeos:
            if not episodio['id'] in existentes:
                conexao.execute('insert into episodio values (?, ?, ?, ?, ?, ?)', 
                                (episodio['id'], episodio['number'], episodio['title'], episodio['description'], episodio['duration'], episodio['published_at']))
        
        print('Processo concluído')

In [6]:
executar()

Página 1 de 20.
Sleep 6.06
Página 2 de 20.
Sleep 5.22
Página 3 de 20.
Sleep 12.01
Página 4 de 20.
Sleep 11.55
Página 5 de 20.
Sleep 9.13
Página 6 de 20.
Sleep 8.87
Página 7 de 20.
Sleep 4.87
Página 8 de 20.
Sleep 7.34
Página 9 de 20.
Sleep 12.89
Página 10 de 20.
Sleep 10.53
Página 11 de 20.
Sleep 8.82
Página 12 de 20.
Sleep 13.95
Página 13 de 20.
Sleep 8.13
Página 14 de 20.
Sleep 8.66
Página 15 de 20.
Sleep 7.59
Página 16 de 20.
Sleep 7.20
Página 17 de 20.
Sleep 6.87
Página 18 de 20.
Sleep 3.20
Página 19 de 20.
Sleep 14.14
Página 20 de 20.
Processo concluído
